1. Numerically simulate a Rayleigh multipath channel. Choose a transmitter location (xt, yt), an array of locations [xr(tm), yr(tm)] modeling a moving receiver, and random coordinates (xn, yn) for Ns scatterers. The receiver should at least a few wavelengths in total and a fraction of a wavelength in each time step. Model the electric field intensity at the receiver using E =
PNs n=1 Rne−jkrn /rn where Rn is a reflection coefficient and rn is the total path distance from the transmitter to the nth scatterer and then to the receiver. For a simple channel model, Rn = 1, or the reflection coefficient can be uniformly distributed with magnitude between zero and one.

(a) Plot the time history of the received power on a dB scale